In [5]:
import os
import pandas as pd
import numpy as np
import json
import re
import gensim
import nltk
import requests
import datetime
import seaborn as sns

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from datetime import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import re



today = datetime.today()

# Monster Jobs

In [6]:
#Catergory Getter
headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '\
           'AppleWebKit/537.36 (KHTML, like Gecko) '\
           'Chrome/75.0.3770.80 Safari/537.36'}
Category_URL_List = ['https://www.monster.com.ph/search/it-jobs',
 'https://www.monster.com.ph/search/manufacturing-engineering-r-d-jobs',
 'https://www.monster.com.ph/search/pharmaceutical-biotechnology-jobs',
 'https://www.monster.com.ph/search/telecom-isp-jobs',
 'https://www.monster.com.ph/search/health-care-jobs',
 'https://www.monster.com.ph/search/engineering-mechanical-automotive-industrial-jobs',
 'https://www.monster.com.ph/search/sales-engineering-technical-it-jobs',
 'https://www.monster.com.ph/search/marine-services-jobs',
 'https://www.monster.com.ph/search/engineering-electrical-jobs',
 'https://www.monster.com.ph/search/engineering-electronics-communication-jobs',
 'https://www.monster.com.ph/search/oil-gas-jobs',
 'https://www.monster.com.ph/search/engineering-chemical-jobs',
 'https://www.monster.com.ph/search/engineering-environmental-health-safety-jobs',
 'https://www.monster.com.ph/search/architecture-interior-design-jobs',
 ]

Category_URL_List




['https://www.monster.com.ph/search/it-jobs',
 'https://www.monster.com.ph/search/manufacturing-engineering-r-d-jobs',
 'https://www.monster.com.ph/search/pharmaceutical-biotechnology-jobs',
 'https://www.monster.com.ph/search/telecom-isp-jobs',
 'https://www.monster.com.ph/search/health-care-jobs',
 'https://www.monster.com.ph/search/engineering-mechanical-automotive-industrial-jobs',
 'https://www.monster.com.ph/search/sales-engineering-technical-it-jobs',
 'https://www.monster.com.ph/search/marine-services-jobs',
 'https://www.monster.com.ph/search/engineering-electrical-jobs',
 'https://www.monster.com.ph/search/engineering-electronics-communication-jobs',
 'https://www.monster.com.ph/search/oil-gas-jobs',
 'https://www.monster.com.ph/search/engineering-chemical-jobs',
 'https://www.monster.com.ph/search/engineering-environmental-health-safety-jobs',
 'https://www.monster.com.ph/search/architecture-interior-design-jobs']

In [37]:
#URL Page Gatherer
isNext = True
isNext_Page = 1
isPer_Page = []
kekw = 8
URL = Category_URL_List[kekw]
page=requests.get(URL,headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
while isNext:
    page=requests.get(URL,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    for div in soup.findAll('div', {"class": "srp-navigation"}):
        Test = div.find_all("button")
    if len(Test) > 1:
        isOutput = Test[1].get_text().replace('\n','').replace(' ','').replace('\t','')
        if isOutput == "Next":
            #print(isOutput)
            isNext_Page = isNext_Page + 1
            #print(isNext_Page)
            URL = Category_URL_List[kekw] + "-{}?".format(isNext_Page)
            print(URL)
            isPer_Page.append(Category_URL_List[kekw] + "-{}?".format(isNext_Page))
        else:
            #print(isOutput)
            #print(isNext_Page)
            isNext = False
    elif (len(Test) == 1):
        isOutput = Test[0].get_text().replace('\n','').replace(' ','').replace('\t','')
        if isOutput == "Next":
            #print(isOutput)
            #print(isNext_Page)
            URL = Category_URL_List[kekw] + "-{}?".format(isNext_Page)
            print(URL)
            isPer_Page.append(Category_URL_List[kekw] + "-{}?".format(isNext_Page))
            isNext_Page = isNext_Page + 1
        elif isOutput == "Previous":
            #print(Test[0].get_text().replace('\n',''))
            #print ("Test")
            #print(isNext_Page)
            isNext = False
    else:
        break

https://www.monster.com.ph/search/engineering-electrical-jobs-1?
https://www.monster.com.ph/search/engineering-electrical-jobs-2?
https://www.monster.com.ph/search/engineering-electrical-jobs-4?
https://www.monster.com.ph/search/engineering-electrical-jobs-5?
https://www.monster.com.ph/search/engineering-electrical-jobs-6?
https://www.monster.com.ph/search/engineering-electrical-jobs-7?
https://www.monster.com.ph/search/engineering-electrical-jobs-8?
https://www.monster.com.ph/search/engineering-electrical-jobs-9?
https://www.monster.com.ph/search/engineering-electrical-jobs-10?
https://www.monster.com.ph/search/engineering-electrical-jobs-11?
https://www.monster.com.ph/search/engineering-electrical-jobs-12?
https://www.monster.com.ph/search/engineering-electrical-jobs-13?
https://www.monster.com.ph/search/engineering-electrical-jobs-14?
https://www.monster.com.ph/search/engineering-electrical-jobs-15?
https://www.monster.com.ph/search/engineering-electrical-jobs-16?
https://www.monste

In [38]:
#Gathering Data 
job_title_list = []
job_employment_type_list = []
job_jobLocation_list = []
job_dateposted_list = []
job_desc_list = []
job_salary_list = []
job_location_list = []
job_type_list = []
comapny_name_list = []
job_years_list = []
job_education_list = []

#The number of pages
for kek in range(0,len(isPer_Page)): ##len(isPer_Page)
    print("Page Number: ",kek)
    #Getting the Job URL per page
    URL = isPer_Page[kek]
    page=requests.get(URL,headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    jobURL_List = []
    i=0
    for div in soup.find_all('div', class_='job-tittle'):
        isSponsored = div.find_all('div', class_='sponsr') 
        if(len(isSponsored)):
            print("do nothing")
        else:
            for h3 in div.find_all('h3', class_='medium'):
                 for a in h3.find_all('a', href=True):
                    jobURL_List.append(a['href'])
                    i=i+1

    #Getting the Info of the JOB                

    for i in range(len(jobURL_List)):
        Monster_URL = jobURL_List[i]
        Monster_page=requests.get(Monster_URL,headers=headers)
        Monstersoup = BeautifulSoup(Monster_page.content, 'html.parser')
        for div in Monstersoup.findAll('div', {"class": "job-tittle detail-job-tittle jd-mt-0"}):
                Job_Title = div.findAll('h1')
                Company_name = div.findAll('a')
                Office_Location = div.findAll('span',{"class": "loc jd-loc"} )
                Salary = div.findAll('span',{"class": "package"} )
                Years_WE = div.findAll("span",{"class": "loc"})
        Job_location =  Monstersoup.findAll("div",{"class": "posted-update pl5 w100"})
        Date_posted = Monstersoup.findAll('span',{"class": "posted seprator pLR-10"} )
        Job_Description = Monstersoup.findAll('div', {"class": "card-body card-body-apply pd20"})
        Job_detail = Monstersoup.findAll('div', {"class": "job-detail-list"})
        #Job_Education = Monstersoup.findAll('a', {"href": "https://www.monster.com.ph/search/bachelors-degree-jobs"})
        Job_Education = Monstersoup.findAll('div', {"class": "job-detail-list"})

        #Data Appending
        try:
            job_title_list.append(Job_Title[0].get_text().replace('\n',' '))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            comapny_name_list.append("Error")
            

        try:
            comapny_name_list.append(Company_name[0].get_text().replace('\n',' '))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            comapny_name_list.append("Not Specified")

        try:
            job_employment_type_list.append(Job_detail[0].get_text().replace('\n',' '))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            job_employment_type_list.append("Not Specified")
        try:    
            job_type_list.append(Job_detail[1].get_text().replace('\n',' '))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            job_type_list.append("Not Specified")

        try:
            job_jobLocation_list.append(Office_Location[0].get_text().replace('\n',' '))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            job_jobLocation_list.append("Not Specified")
        
        try:
            job_dateposted_list.append(Date_posted[0].get_text().replace('\n',' '))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            job_dateposted_list.append("Not Specified")
        
        try:            
            job_desc_list.append(Job_Description[0].get_text().replace('\n',' '))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            job_desc_list.append("Not Specified")
        
        try:
            job_salary_list.append(Salary[0].get_text().replace('\n',' '))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            job_salary_list.append("Not Specified")

        try:
            job_location_list.append(Job_location[0].get_text().replace('\n',' '))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            job_location_list.append("Not Specified")

        try:
            job_years_list.append(Years_WE[1].get_text().replace('\n',''))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            job_years_list.append.append("Not Specified")

        try:
            job_education_list.append(Job_Education[5].get_text().replace('\n',''))
        except:
            #print("An exception occurred") 
            #print(Job_Title[0].get_text().replace('\n',' '))
            #print(jobURL_List[i]) 
            job_education_list.append("Not Specified")

        jobs_data={'Website:': "Monster PH",
                   'Job Title': job_title_list, 
                   'Category': job_type_list,
                   'Company': comapny_name_list,
                   'Date Posted': job_dateposted_list, 
                   'Location': job_location_list,
                   'Status': job_employment_type_list, 
                   'Salary': job_salary_list,
                   'Education': job_education_list,
                   'Years of Work Expirience': job_years_list,
                   'Job Description': job_desc_list,
                   'Office Location': job_jobLocation_list, 
                   }
        monster_jobs_df = pd.DataFrame(data=jobs_data)

    

data = monster_jobs_df.to_json(orient='records')
parsed = json.loads(data)
json.dumps(parsed, indent=4) 
with open('monster_ph_engineering-engineering-electrical.json', 'w') as json_file:
    json.dump(parsed, json_file)
#Website	Job Title	Category	Company	Date Posted	Location	Status	Salary	Education	Years of Work Expirience	Job Description

Page Number:  0
Page Number:  1
Page Number:  2
Page Number:  3
Page Number:  4
Page Number:  5
Page Number:  6
Page Number:  7
Page Number:  8
Page Number:  9
Page Number:  10
Page Number:  11
Page Number:  12
Page Number:  13
Page Number:  14
Page Number:  15
Page Number:  16
Page Number:  17


ChunkedEncodingError: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))

In [34]:
monster_jobs_df_json = pd.read_json(r'monster_ph_health-care.json')
monster_jobs_df_json

,Website:,Job Title,Category,Company,Date Posted,Location,Status,Salary,Education,Years of Work Expirience,Job Description,Office Location
0,Monster PH,USRN- 150K Sign on Bonus- Work From Home - Cai...,Industry: ITES / BPO,Not Specified,Posted On: 4 hours ago,"Permanent Job, ...",Employment Types: Full time,Not Spec...,Education:Bachelors/ Degree,1-...,Job Description TeleTech is on the search ...,Oth...
1,Monster PH,Customer Service Representative - Premium Telc...,Industry: ITES / BPO,Not Specified,Posted On: 4 hours ago,"Permanent Job, ...",Employment Types: Full time,Not Spec...,Education:Bachelors/ Degree,1-...,Job Description TeleTech is on the search ...,Wes...
2,Monster PH,US Registered Nurses - 150K Sign on Bonus- Wor...,Industry: ITES / BPO,Not Specified,Posted On: 4 hours ago,"Permanent Job, ...",Employment Types: Full time,Not Spec...,Education:Bachelors/ Degree,1-...,Job Description TeleTech is on the search ...,Oth...
3,Monster PH,Pioneer | Technical Support | Chinese [Mandari...,Industry: ITES / BPO,Not Specified,Posted On: 5 hours ago,"Permanent Job, ...",Employment Types: Full time,Not Spec...,Education:Bachelors/ Degree,0-...,Job Description TeleTech is on the search ...,Met...
4,Monster PH,Customer Service Representative- Dumaguete,Industry: ITES / BPO,Not Specified,Posted On: 4 hours ago,"Permanent Job, ...",Employment Types: Full time,Not Spec...,Education:Bachelors/ Degree,0-...,Job Description TeleTech is on the search ...,Cen...
...,...,...,...,...,...,...,...,...,...,...,...,...
1715,Monster PH,Sr. Talent Acquisition Manager - iloilo,Industry: Recruitment / Staffing / RPO,eTeam Workforce Pte Ltd,Posted On: 27 days ago,Permanent Job ...,Employment Types: Full time,Not Spec...,Not Specified,No...,Job Description Job Description :Position ...,Phi...
1716,Monster PH,National Consultant for the Development of Ris...,Industry: Other,United Nations Development Programme,Posted On: a month ago,Permanent Job ...,Employment Types: Part time,Not Spec...,Not Specified,No...,Job Description Job Description :Project N...,Phi...
1717,Monster PH,Platform User Experience (UX) and User Interfa...,Industry: Other,United Nations Development Programme,Posted On: a month ago,Permanent Job ...,Employment Types: Part time,Not Spec...,Not Specified,No...,Job Description Job Description :Enhanced ...,Phi...
1718,Monster PH,Consultant to Conduct a Technical Needs Assess...,Industry: Other,United Nations Development Programme,Posted On: 6 days ago,Permanent Job ...,Employment Types: Part time,Not Spec...,Not Specified,No...,Job Description Job Description :Consultan...,Phi...
